<a href="https://colab.research.google.com/github/tamaUdon/tempwriter/blob/main/model/train/train_shadow_alphabet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Shadow Alphabetデータセットの拡張
ref. https://www.tensorflow.org/tutorials/images/data_augmentation?hl=ja


### セットアップ

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers

### データセットをダウンロードする

In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

num_classes = metadata.features['label'].num_classes
print(num_classes)

### Keras前処理レイヤーを利用する

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.RandomRotation(0.2), # 回転
  layers.RandomBrightness(factor=0.2) # 輝度
  layers.Rescaling(1./255) # scale
])

注意: 上記のリスケーリングレイヤーは、ピクセル値を [0,1] の範囲に標準化します。代わりに [-1,1] を用いる場合には、tf.keras.layers.Rescaling(1./127.5, offset=-1) と記述します。

### 前処理レイヤーをデータセットに適用する

In [ ]:
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, shuffle=False, augment=False):
  # Resize and rescale all datasets.
  ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets.
  ds = ds.batch(batch_size)

  # Use data augmentation only on the training set.
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefetching on all datasets.
  return ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

# TODO: driveに保存する

## Shadow Alphabetモデルの転移学習
ref. https://developers.google.com/mediapipe/solutions/vision/gesture_recognizer/customize


### セットアップ

In [ ]:
pip install --upgrade pip
pip install mediapipe-model-maker

In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

### データセットを取得する

In [ ]:
!wget https://storage.googleapis.com/mediapipe-tasks/gesture_recognizer/rps_data_sample.zip
!unzip rps_data_sample.zip
dataset_path = "rps_data_sample"

### データセットをロードする

ロード時に、MediaPipe Handsから事前にパッケージ化された手検出モデルを実行して、画像から手のランドマークを検出します。<br>
<br>
手が検出されない画像はデータセットから除外されます。<br>
結果として得られるデータセットには、画像自体ではなく、各画像から抽出された手のランドマークの位置が含まれます。<br>
<br>
このHandDataPreprocessingParamsクラスには、データ読み込みプロセス用の2つの構成可能なオプションが含まれています。<br>
<br>
shuffle: データセットをシャッフルするかどうかを制御するブール値。デフォルトは true<br>
min_detection_confidence: 手を検出するための信頼しきい値を制御する 0 ～ 1 間のfloat<br>

In [ ]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
# 8:1:1=train:val:test
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

### モデルをトレーニングする

In [ ]:
hparams = gesture_recognizer.HParams(export_dir="exported_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

### モデルのパフォーマンスを評価する

In [ ]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

### モデルの保存

In [ ]:
# TODO: tflite形式でなくtf形式で保存する
model.export_model()
!ls exported_model

files.download('exported_model/gesture_recognizer.task')